In [1]:
import scanpy as sc

dr = sc.read_h5ad('/storage/lingyuan2/STATES_data/withDR.h5ad')


In [ ]:
points_df = dr.uns['points_df']
points_df['group'] = points_df['feature_name'].str.split('_').str[-1]
points_df

In [ ]:
import pandas as pd
import numpy as np

bins = [0, 0.33, 0.66, 1.0]

results = []
for cell_idx, cell_data in points_df.groupby('cell_idx'):
    cell_data['DR_bin'] = pd.cut(cell_data['DR'], bins=bins, include_lowest=True, right=False)
    for bin_range, bin_data in cell_data.groupby('DR_bin'):
        rbRNA_counts = bin_data[bin_data['group'] == 'rbRNA'].shape[0]
        ntRNA_counts = bin_data[bin_data['group'] == 'ntRNA'].shape[0]
        if (rbRNA_counts + ntRNA_counts) > 0:
            te = rbRNA_counts / (rbRNA_counts + ntRNA_counts)
        else:
            te = np.nan
        results.append({
            'cell_idx': cell_idx,
            'DR_bin': bin_range,
            'TE': te,
            'rbRNA_counts': rbRNA_counts,
            'ntRNA_counts': ntRNA_counts
        })

te_by_dr_bin = pd.DataFrame(results)
te_by_dr_bin

In [ ]:
te_by_dr_bin.to_csv('/storage/lingyuan2/STATES_data/te_by_dr_bin_cell.csv', index=False)
te_by_dr_bin